In [5]:
from osgeo import gdal
import os, shutil
import numpy as np
gdal.UseExceptions()

# This script is to create and submit mulctiple Sx calculations

work_dir='/glade/u/home/hongli/scratch/2020_11_29discretization_error/discretize/yampa'
script_dir='/glade/u/home/hongli/github/2020_11_29discretization_error/radiation_wind_functions'
script_file='step5_raw_Sx_function.py'

row_interval=70 # use 38BG memory
col_interval=70
buf_grid_window=100 #130, 300 # unit:m
dem_nodata=-9999

# =======================input and output =============================
dem_raster=os.path.join(work_dir, 'dem_crop.tif')
dem_buf_raster=os.path.join(work_dir, 'dem_crop_buf.tif') # extend for dem raster Sx
opath=os.path.join(work_dir, 'step5_raw_Sx')
if not os.path.exists(opath):
    os.makedirs(opath)
command_folder=os.path.join(work_dir, 'step5_raw_Sx_command')
if os.path.exists(command_folder):
    shutil.rmtree(command_folder) 
os.makedirs(command_folder)
shutil.copyfile(os.path.join(script_dir, script_file), os.path.join(command_folder,script_file))
 
#====================================================
# read raw raster [ELEVATION]
print('read raw data')
r = gdal.Open(dem_raster)
band = r.GetRasterBand(1) #bands start at one
elev = band.ReadAsArray().astype(np.float)
mask = (elev==dem_nodata)
(ny,nx)=np.shape(elev)

print('write command file')
os.chdir(command_folder)

if ny%row_interval==0:
    row_job_num=ny//row_interval
else:
    row_job_num=ny//row_interval+1
if nx%col_interval==0:
    col_job_num=nx//col_interval
else:
    col_job_num=nx//col_interval+1 

job_id = 0 
for i in range(row_job_num):
    for j in range(col_job_num):

        row_start = i*row_interval
        col_start = j*col_interval
        if (ny%row_interval!=0) and (i==row_job_num-1):
            row_end=row_start+ny%row_interval
        else:
            row_end = (i+1)*row_interval
        if (nx%col_interval!=0) and (j==col_job_num-1):
            col_end = col_start+nx%col_interval
        else:
            col_end = (j+1)*col_interval
            
        mask_subset = mask[row_start:row_end,col_start:col_end]
        if not mask_subset.all()==True:        
            job_id = job_id+1
            print('Job '+str(job_id))
            
            # create command file
            command_file='qsub_Job'+str(job_id)+'_Row'+str(row_start)+'_'+str(row_end)\
            +'_Col'+str(col_start)+'_'+str(col_end)+'.sh'
            command_file_path = os.path.join(command_folder,command_file)
            if os.path.exists(command_file_path):
                os.remove(command_file_path)
                
            with open(command_file_path,'w') as f:
                f.write('#!/bin/bash\n')
                f.write('#PBS -N SxJob'+str(job_id)+'\n')
                f.write('#PBS -A P48500028\n')
                f.write('#PBS -q regular\n')
                f.write('#PBS -l walltime=00:25:00\n')
                f.write('#PBS -l select=1:ncpus=1:mpiprocs=1\n') 
                f.write('#PBS -j oe\n\n')
                f.write('export TMPDIR=/glade/scratch/hongli/tmp\n')
                f.write('mkdir -p $TMPDIR\n\n')                
                f.write('module load peak_memusage\n\n') # monitor peak memory usage   
                # Note: cheyenne usable memory per compute node is 45GB.
                f.write('peak_memusage.exe /glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/bin/python '\
                        +os.path.join(command_folder,script_file)+' '+dem_raster+' '+dem_buf_raster+' '\
                        +str(row_start)+' '+str(row_end)+' '\
                        +str(col_start)+' '+str(col_end)+' '\
                        +str(buf_grid_window)+' '+opath)
                
            # submit job
            os.system('chmod 740 '+command_file)
            os.system('qsub '+command_file)

os.chdir(work_dir)        
print('Total job number = '+str(job_id))
print('Done')

read raw data
write command file
Job 1
Job 2
Job 3
Job 4
Job 5
Job 6
Job 7
Job 8
Job 9
Job 10
Job 11
Job 12
Job 13
Job 14
Job 15
Job 16
Job 17
Job 18
Job 19
Job 20
Job 21
Job 22
Job 23
Job 24
Job 25
Job 26
Job 27
Job 28
Job 29
Job 30
Job 31
Job 32
Job 33
Job 34
Job 35
Job 36
Job 37
Job 38
Job 39
Job 40
Job 41
Job 42
Job 43
Job 44
Job 45
Job 46
Job 47
Job 48
Job 49
Job 50
Job 51
Job 52
Job 53
Job 54
Job 55
Job 56
Job 57
Job 58
Job 59
Job 60
Job 61
Job 62
Job 63
Job 64
Job 65
Job 66
Job 67
Job 68
Job 69
Job 70
Job 71
Job 72
Job 73
Job 74
Job 75
Job 76
Job 77
Job 78
Job 79
Job 80
Job 81
Job 82
Job 83
Job 84
Job 85
Job 86
Job 87
Job 88
Total job number = 88
Done


In [1]:
import rasterio as rio
import matplotlib.pyplot as plt 
dem_origin_file ='/glade/u/home/hongli/scratch/2020_11_29discretization_error/discretize/source_data/MERIT_Hydro_dem_NLDAS.tif'

with rio.open(dem_origin_file) as ff:
    in_raster_value  = ff.read(1)
    in_raster_mask = ff.read_masks(1)
    gt = ff.transform

dx = gt[0]
dy = -gt[4]

In [3]:
dx,dy

(0.0008333333333333467, 0.0008333333333333467)